# Сходство текстов

### Задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

In [1]:
import re 
from collections import Counter
my_file = open('sentences.txt')

#### Создадим словарь для которого все различные слова текста это индексы, а кол-во их повторений это значения. Посмотрим какие слова чаще всего встречаются.

In [2]:
tokens = []
sentences = []
lines_counter = 0
for line in my_file:
    newline = re.split('[^a-z]',line.lower())
    token = [el for el in newline if el != '']
    tokens += token
    sentences.append(token)
#word_dict = dict([(x,tokens.count(x)) for x in set(tokens)])
word_dict = dict(Counter(tokens))
print (Counter(tokens).most_common(10))
my_file.close()

[('the', 20), ('of', 19), ('to', 14), ('and', 14), ('a', 13), ('in', 11), ('cat', 10), ('x', 9), ('apple', 8), ('os', 8)]


#### Cоздадим матрицу входимости слов в предложения размера n*d  , где  n кол-во предложений в тексте, а d это кол-во слов. 

In [3]:
import numpy as np
my_file = open('sentences.txt')
matrix = []
d = (len(word_dict))
print ('кол-во слов = ',d)
line_num = 0
dict_keys_list = list(word_dict.keys())
for line in my_file:
    line_count = [sentences[lines_counter].count(word) for word in dict_keys_list ]
    lines_counter += 1
    matrix.append(line_count)
np_matrix = np.array(matrix)
print ('размерность матрицы = ',np_matrix.shape)
np.set_printoptions(threshold=np.inf)
np_matrix.tofile('my_last.csv',sep=',',format='%10.1f')
my_file.close()

кол-во слов =  254
размерность матрицы =  (22, 254)


### Посчитаем меру сходства предложений используя формулу :
$$ Cosine distance  = 1 - \frac{(u , v)} {||u||_2 ||v||_2} $$
   Соответсвенно, чем меньше это число тем более похожи предложения

In [4]:
from scipy.spatial.distance import cosine
first_raw = matrix[0] 
cosine_vec = []
for raw in np_matrix[1:]:
    cosine_vec.append(cosine(first_raw,raw))
print ('Вектор мер схожести первого предложения с остальными:\n\n', cosine_vec)
np_cosine = np.array(cosine_vec)

Вектор мер схожести первого предложения с остальными:

 [0.95275444087384664, 0.86447381456421235, 0.89517151632780823, 0.77708871496985887, 0.94023856953328033, 0.7327387580875756, 0.92587506833388988, 0.88427248752843102, 0.90550888174769317, 0.83281653622739416, 0.88047713906656067, 0.83964325485254543, 0.87035925528956715, 0.87401184233025764, 0.94427217874246472, 0.84063618542208085, 0.95664450152379399, 0.94427217874246472, 0.88854435748492944, 0.84275727449171223, 0.82503644694405864]


In [5]:
import codecs
file = codecs.open('Lab1_output.txt', encoding = 'utf-8', mode ='w') 
result = ('Отсортированый по близости расстояний массив индексов предложений = {}'.format(np.argsort(np_cosine)+2))
file.write(result)
file.close()

### Заданое предложение:
In comparison to dogs, cats have not undergone major changes during the domestication process.
### Найденное ближайшее по смыслу предложение:
Domestic cats are similar in size to the other members of the genus Felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).